# 解决思维谜题：”帐篷”

规则和案例可以见：[在线链接](https://cn.puzzle-tents.com/?size=8)

思维游戏“帐篷”的规则很简单:

- 在与树水平或竖直方向相邻的格子里放置帐篷。帐篷与树一一对应。
- 帐篷之间互不相邻（即便是对角也不行）
- 边框外的数字表示每行、每列的帐篷总数。(最后的实例忘记截了...)

-----

输入数据：

1. 图的规模；
2. 树的位置；
3. 行、列外的数字

输出数据：

帐篷的位置

-------------



The rules of the logic game "Tents" are very simple:

- Place tents in squares adjacent horizontally or vertically to trees. Each tent corresponds to a tree.
- Tents cannot be adjacent to each other (not even diagonally).
- Numbers outside the grid indicate the total number of tents in each row and column. (The last example was forgotten to be cut off...)

-----

Input Data:

1. Size of the grid.
2. Positions of the trees.
3. Numbers outside the rows and columns.

Output Data:

Positions of the tents.



In [1]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

def GetNeighbours(N, pos):
    
    neighbours = []
    directions = [[1,0], [-1,0], [0,1], [0,-1]]
    for direction in directions:
        if pos[0] + direction[0] >= N or pos[0] + direction[0] < 0 \
            or pos[1] + direction[1] >= N or pos[1] + direction[1] < 0:
                continue
        neighbours.append((pos[0] + direction[0], pos[1] + direction[1]))
    
    return neighbours

def TentSolver(N, trees, rows, cols):

    if len(rows) != N or len(cols) != N:
        raise Exception("N和输入数据长度不符")

    model = cp.CpModel()
    x = {}
    for i in range(N):
        for j in range(N):
            x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
    
    for tree in trees:
        neighbours = GetNeighbours(N, tree)
        cage = [x[i[0], i[1]] for i in neighbours]
        model.Add(sum(cage) >= 1)
        model.Add(x[tree[0], tree[1]] == 0)
    
    for idx, row in enumerate(rows):
        model.Add(sum([x[i, idx] for i in range(N)]) == row)
    
    for idx, col in enumerate(cols):
        model.Add(sum([x[idx, j] for j in range(N)]) == col)
    
    
    for i in range(N - 1):
        for j in range(N - 1):
            model.Add(sum([x[i, j], x[i + 1, j], x[i, j + 1], x[i + 1, j + 1]]) <= 1)
        

    solver = cp.CpSolver()
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(N):
            for j in range(N):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
        

if __name__ == "__main__":
    # rows = [3,0,2,0,2,0]
    # cols = [1,0,2,1,2,1]
    # trees = [
    #     (1,0),
    #     (2,1),
    #     (3,0),
    #     (3,3),
    #     (3,4),
    #     (4,5),
    #     (5,1)
    # ]
    # TentSolver(6, trees, rows, cols)
    
    rows = [5,2,4,1,4,1,3,2,4,4,3,3,3,3,3]
    cols = [6,1,3,3,2,2,5,3,4,1,2,3,3,1,6]
    trees = [
        (0,5), (0,6), (0,14),
        (1,0), (1,1),(1,2), (1,9), (1,10), (1,13),
        (2,8),
        (3,5),
        (4,0),(4,7),(4,8),(4,11),
        (5,2),(5,5),(5,8),(5,11),(5,14),
        (6,0),(6,11),
        (7,4),(7,5),(7,13),
        (8,5),(8,7),(8,11),
        (9,2),(9,9),(9,13),
        (10,0),
        (11,5),
        (12,1),(12,6),(12,10),(12,11),(12,13),
        (13,8),(13,12),
        (14,1),(14,3),(14,6),(14,7),(14,13)
    ]
    TentSolver(15, trees, rows, cols)

1 0 1 0 1 0 0 1 0 1 0 0 0 1 0 
0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 
0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 
0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 
1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 
0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 
0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 
1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 
1 0 1 0 0 1 0 0 1 0 0 0 1 0 1 

NumConflicts: 0
NumBranches: 262
WallTime: 0.019094



![](../assets/figures/TentExample.png)

![](../assets/figures/TentExample2.png)
